# Cloud Computing

Cloud computing is the use of remote servers on the Internet to store (host), manage and process data rather than a local server or personal computer.   Goals for this module:
- Types of cloud computing resources
- Launching virtual machines
- Setting up virtual machines for computation
- setting up hadoop

## Types of cloud computing resources
There are three general categories of cloud computing resources, listed here in increasing complexity for the end user:
- SaaS: Software as a service.  A modern example would be the suite of Google applications available to us.  Whereas we would previously download a text editor and a spreadsheet program to our computers (and we still may!), we could also use this software in the cloud, e.g., Google docs and Google sheets.
- PaaS: Platform as a service. Often targeted at developers of software who don't want to worry about the environment.  An example of this might be the galaxy product for bioinformatics researchers, https://usegalaxy.org/.  There are providers (e.g. the Texas Advanced Computing Center) that provide web-based servers that researchers can log into, and start programming their workflow / data munging / integrating different python without having to deploy their own galaxy instance.  JupyterHub can also be classified as a PaaS, providing developers a chance to code in python without having to setup their computing environment.
- IaaS: Infrastructure as a service.  Provides computational resources (servers, storage, bandwidth) to an end user.

We will be exploring IaaS in this module.  Idea: there are large data centers distributed around the world that host compute servers and large file servers.  By collocating the resources in a large data center, benefit from economy of scale, and there can be multiple technicians on-site to handle hardware failures.  

There are multiple companies that provide IaaS.  Some big names you ight have heard of:
- Amazon's EC2 (Elastic Compute Cloud)
- Microsoft Azure Cloud 
- Google Cloud

Our hands-on tutorial today will utilize the EC2 ecosystem.  I have no experience with the other cloud providers, though Microsoft and Google are both highly invested in higher-education and research in academia.
![cloud](figures/ec2.gif)

- launch and log into amazon management console
- There are (as of the writing of this tutorial), various solutions:
    - launch a virtual machine with EC2 (this is what we will be exploring)
    - building web application with Elastic Beanstalk (pre-configured solutions for ava, .NET, PHP, Node.js, Python, Ruby, Go, and Docker)
    - build using virual servers with Lightsail (seems to be a simplified EC2 interface)
    - connect with IoT device with AWS IoT.  (maybe you have a smart fridge at home that you want to do some analysis on?)
    - pre-configured servers for development project
    
Lets work with the full EC2 interface.

### Key-Pairs
The first step is to setup keys to access (via ssh) any instance that you setup.  The PEM files can be generated via the EC2 key-pairs tab.  PEM (Privacy Enhanced Mail) is a Base64 encoded certificate 

### Instance
Next, we launch the instance.  The first step is to choose an image.  If you have a flavor of linux you a familiar with, you can choose that.  Many enterprise solutions lean towards Red Hat Enterprise Linux (RHEL) or their related variants due to the availability of support contracts.  There are also windows servers, some preconfigured deep-learning images (which already has TensorFlow, Keras, PyTorch) along with accelerator tools pre-installed.  (These deep learning VMs are unfortunately not part of the free tier, but we can spend a bit of effort to arrive at a similar free solution).  Some are pre-configured for webhosting, with Apache, Mysql, Php and Ruby preinstalled.  

For now, let's pick the fourth one down, Red Hat Enterprise Linux 7.6 (HVM), SSD Volume Type.  x86.  

Next, one selects the type of resources you want the virtual machine to have access to.  There is only one option for the free tier: 1 virtual computing core with access to 1 GiB of memory.  Storage is via the amazon cloud service EBS (Elastic Block Store), and low-to moderate network performance.  The non-free tier includes choices for the number of virtual CPUS, Memory, GPU, attached SSD storage, high throughput networking.   There are other security options and file access options one can select. 

### Connection

To connect to the instance, use
``` bash
ssh -i mykey.pem ec2-user@ip-address
```

### Setting up the environment

- Lets setup Anaconda.  We can download this file: https://repo.anaconda.com/archive/Anaconda3-5.3.1-Linux-x86_64.sh
- and execute using
``` bash
bash Anaconda3-5.3.1-Linux-x86_64.sh
```
and follow instructions for installation.
- we want to be able to launch jupyter notebooks.  First, we need to config:
```bash
jupyter notebook --generate-config
```
- and create a cert
```bash
mkdir certs
cd certs
sudo openssl req -x509 -nodes -days 365 -newkey rsa:1024 -keyout mycert.pem -out mycert.pem
```
- and add this cert to the jupyter config file
```bash
vi ../.jupyter/jupyter_notebook_config.py
```
and add in the lines
```bash
c = get_config()
# Notebook config this is where you saved your pem cert
c.NotebookApp.certfile = u'/home/ec2-user/certs/mycert.pem' 
# Run on all IP addresses of your instance
c.NotebookApp.ip = '*'
# Don't open browser by default
c.NotebookApp.open_browser = False  
# Fix port to 8888
c.NotebookApp.port = 8888
```
- update the security group for the EC2 instance
- start and connect to jupyter notebook

# Distributed File Systems

One of the big benefits for using cloud computing, is the ability to scale the analysis we are doing to larger data sets.  The personal computers we are using are limited by computational speed/throughput, memory and bandwidth constraints, i/o constraints (size + speed).  Lets explore how we can use cloud computing alleviate i/o constraints.

It's easy to picture the increased i/o storage capabilities.  On a small scale, one might have a raided network attached storage (NAS), e.g., https://www.amazon.com/NETGEAR-ReadyNAS-Diskless-Personal-Processor/dp/B015S0Y50C.  On a larger scale, one could fit many more storage drives in a system that spans multiple racks, e.g., https://www.digitalengineering247.com/article/storage-is-key-to-hpc/ , or even tape, https://spectralogic.com/industry-solutions/high-performance-computing/.  The ability to store more information is not useful unless we can access the information quickly.  

SSDs have certainly come a long way - the SSD on my macbook pro is apparently capable of 2Gb/s writing and 3Gb/s reading, which is an order of magnitude faster than what was available 2 years ago. That said, if you have a file that is say 100Gb, it would still take you 30 seconds or so to read it in.  The solution is a "distributed" file system - one stores and reads a single file across many devices.  There are several accepted distributed file systems presently:
- GPFS (Global Parallel File System), a proprietary system from IBM.  It is used by many of the larger commercial companies because it scales across various media (tape <-> ssd <-> dram) and has a solid support infrastructure
- LUSTRE, an open source distributed file system that is commonly found in HPC centers, and supports over 50% of the world's largest supercomputers.  The MTU supercomputing system has a lustre server.
- HDFS (Hadoop Distributed File System), an open source variant that stores data on commodity machines.  It is primarily targeted for storing data on the node where the computation is done.  This is the variant we will explore, in preparation for the map-reduce lab on thursday.

# HDFS

HDFS is a distributed, scalable, and portable file system written in Java for the Hadoop framework.  We will be able to leverage the Hadoop functionality within Python using libraries, such as mrjobs.  Here is a brief video on HDFS and MapReduce, https://www.youtube.com/watch?v=GJYEsEEfjvk

## Single node
We will stick with a single node first to illustrate the basics of HDFS.  First, we need to install java,
```bash
sudo yum install java-1.8.0-openjdk-devel.x86_64
```
Add the following two lines to the .bashrc
```bash
export JAVA_HOME=/usr/lib/jvm/java-1.8.0-openjdk   
export PATH=$JAVA_HOME/bin:$PATH
```
and source your bashrc
```bash
source ~/.bashrc
```
test your java installation by typing:
```bash
java -version
```

Lets try using downloading and using the hadoop binaries:
```bash
wget http://apache.mirrors.tds.net/hadoop/common/hadoop-3.1.1/hadoop-3.1.1.tar.gz
```
and un-tar and copy to the main system:
```bash
sudo tar zxvf hadoop-3.1.1.tar.gz -C /usr/local
```
for convenience, lets sim-link the directory name to hadoop
```bash
cd /usr/.ib
sudo ln -sf hadoop-3.1.1 hadoop
```
and add the following lines to the .bashrc file
```bash
export HADOOP_HOME=/usr/local/hadoop
export PATH=$PATH:$HADOOP_HOME/bin
export HADOOP_CONF_DIR=$HADOOP_HOME/etc/hadoop
```
and re-source it
```bash
source ~/.bashrc
```
We should now be set to test out a simple HDFS (single node) on our VM.

## Multiple nodes

If we want to set up multiple hadoop data nodes, then we need to configure a master node and then multiple slave data nodes.  The following configuration files control the hadoop setup in $HADOOP_CONF_DIR:
- core-site.xml
- yarn-site.xml (yarn: yet another resource manager)
- hadoop-env.sh
- hdfs-site.xml

as well as specification of the master snd data node hostnames.  Interested readers are directed to:
https://myawsjourney.wordpress.com/2018/01/16/how-to-setup-a-hadoop-v-3-0-0-cluster-using-aws-ec2/